In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import multiprocessing as mp
import os
import cv2
import pickle
import random
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tqdm import tnrange
#Check GPU support, please do activate GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def plot_training_progress(losses, accuracies):
    """
    Plots training loss and accuracy over epochs.

    Args:
        losses (list): List of epoch losses.
        accuracies (list): List of epoch accuracies.
    """
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 5))

    # Create a plot for the loss
    ax1 = plt.gca()  # Get current axis
    ax1.plot(epochs, losses, label='Loss', color='blue', marker='o')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.grid()

    # Create a second y-axis for accuracy
    ax2 = ax1.twinx()  # Create a twin axis sharing the same x-axis
    ax2.plot(epochs, accuracies, label='Accuracy', color='orange', marker='o')
    ax2.set_ylabel('Accuracy', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')

    # Combine legends
    lines, labels = ax1.get_legend_handles_labels()  # Get the loss labels
    lines2, labels2 = ax2.get_legend_handles_labels()  # Get the accuracy labels
    ax1.legend(lines + lines2, labels + labels2)  # Combine both legends

    plt.tight_layout()
    plt.show()
def extract_sample_normal(n_way, n_support, n_query, datax, datay,K):

  sample = []
  sample_indexes = []
  Classes=[]
  for cls in K:
    datax_cls = datax[datay == cls]
    perm = np.random.permutation(len(datax_cls))
    sample_cls = datax_cls[perm[:n_support + n_query]]
    sample.append(sample_cls)
    sample_indexes.append(perm[:n_support + n_query])

    Classes.append(cls)
  sample = np.array(sample)
  sample_indexes = np.array(sample_indexes)
  sample = torch.from_numpy(sample).float()
  sample1 = sample.permute(0,1,4,2,3)

  return({
      'images': sample1,
      'n_way': n_way,
      'n_support': n_support,
      'n_query': n_query,
      'Index':sample_indexes,
      'Classes':Classes
      })
def extract_sample_AIS(n_way, n_support, n_query, datax, datay,K,IP_df):

  sample = []
  #--------------Apply algorithm 3a for instance sampling---------------------------------------
  Ordered_classes_samples=Algo_3a(K,datax,datay,n_way,n_support,n_query,IP_df)
  sample=[]
  sample_indexes = []
  Classes=[]
  for i in Ordered_classes_samples:
    datax_cls = datax[datay == i[0]]
    Np_idx=np.array(i[1])
    sample_cls = datax_cls[Np_idx]  # Get selected indexes
    sample.append(sample_cls)
    sample_indexes.append(Np_idx)
    Classes.append(i[0])


  sample = np.array(sample)
  #--------------indices of  samples---------------------------------------------------
  sample_indexes = np.array(sample_indexes)
  #------------------------------------------------------------------------------------
  sample = torch.from_numpy(sample).float()
  sample1 = sample.permute(0,1,4,2,3)

  return({
      'images': sample1,
      'n_way': n_way,
      'n_support': n_support,
      'n_query': n_query,
      'Index':sample_indexes,
      'Classes':Classes
      })
#----------Function to initialize Instance potential matrix-----------------
def Make_Instance_Potential_Matrix(datax,datay):
  #-----------Total Classes---------------------
  Tot_Classes = datay
  Len_Tot_Classes=len(Tot_Classes)

  #-----------Total Samples---------------------
  Tot_Samples = datax

  #-------------no of samples in each class------------------------
  unique, counts = np.unique(Tot_Classes, return_counts=True)
  distinct_counts = list(zip(unique, counts))

  class_indices = []
  sample_indices = []
  probabilities = []

  for i in distinct_counts:
    class_indices.extend([str(i[0])] * i[1])
    sample_indices.extend(range(i[1]))
    probabilities.extend([float(1/Len_Tot_Classes)] * i[1])

  class_indices=np.array(class_indices)
  sample_indices=np.array(sample_indices)
  probabilities=np.array(probabilities)
  IP_df = pd.DataFrame({
    'class_index': class_indices.astype(str),
    'sample_index': sample_indices.astype(int),
    'probability': probabilities.astype(float)
})

  return IP_df
#---------------------Algorithm for instance sampling------------------------------
def Algo_3a(K, datax, datay, n_way, n_support, n_query, IP_df):
    # Filter for relevant classes and calculate the number of samples required per class
    filtered_df = IP_df[IP_df['class_index'].isin(K)]
    n_samples_per_class = n_support + n_query

    # Select the top samples for each class using `nlargest` directly
    top_Sam_per_class = (
        filtered_df.sort_values(['class_index', 'probability'], ascending=[True, False])
        .groupby('class_index', group_keys=False)
        .head(n_samples_per_class)[['class_index', 'sample_index', 'probability']]
    )

    # Use pivot table or dictionary comprehension to create the output in desired format
    samples_indices = {
        class_idx: top_Sam_per_class[top_Sam_per_class['class_index'] == class_idx]['sample_index'].tolist()
        for class_idx in top_Sam_per_class['class_index'].unique()
    }
    # Return classes in order along with their sample indices
    ordered_classes_with_samples = list(samples_indices.items())

    return ordered_classes_with_samples

#------------------------------update graph weight by misclassified query points-----------------------------------------------
def calculate_mismatch_probabilities(softmax_probs_true, y_hat, target_inds, n_way, n_query, IP_df, x_query_Indices, Alpha, Tau, Ind_Classes):
    # Flatten required tensors
    x_query_Indices = x_query_Indices.flatten()
    softmax_probs_query_flatten = softmax_probs_true.flatten()

    # Apply mismatch probability formula
    updated_softmax_query_flatten = (softmax_probs_query_flatten ** Tau) * torch.exp(Alpha * (1 - softmax_probs_query_flatten))

    # Convert to NumPy for DataFrame update
    updated_softmax_probs_query_flatten_numpy = updated_softmax_query_flatten.detach().cpu().numpy()
    result_Ind_Classes = [element for element in Ind_Classes for _ in range(n_query)]

    # Update DataFrame in a loop
    for i, sample_index in enumerate(x_query_Indices):
        IP_df.loc[
            (IP_df['sample_index'] == sample_index) & (IP_df['class_index'] == result_Ind_Classes[i]),
            'probability'
        ] = updated_softmax_probs_query_flatten_numpy[i]

    return IP_df
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

def load_protonet_conv(**kwargs):
    """
    Loads the prototypical network model
    Arg:
        x_dim (tuple): dimension of input image
        hid_dim (int): dimension of hidden layers in conv blocks
        z_dim (int): dimension of embedded image
    Returns:
        Model (Class ProtoNet)
    """
    x_dim = kwargs['x_dim']
    hid_dim = kwargs['hid_dim']
    z_dim = kwargs['z_dim']

    def conv_block(in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

    encoder = nn.Sequential(
        conv_block(x_dim[0], hid_dim),
        conv_block(hid_dim, hid_dim),
        conv_block(hid_dim, hid_dim),
        conv_block(hid_dim, hid_dim),
        conv_block(hid_dim, hid_dim),
        conv_block(hid_dim, z_dim),
        Flatten()
    )

    # Initialize weights using Kaiming Normal initialization
    for m in encoder.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Linear):  # If there are linear layers
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    return ProtoNet(encoder)
class ProtoNet(nn.Module):
  def __init__(self, encoder):
    """
    Args:
        encoder : CNN encoding the images in sample
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
    """
    super(ProtoNet, self).__init__()
    self.encoder = encoder.to(device)

  def set_forward_loss(self, sample):
    """
    Computes loss, accuracy and output for classification task
    Args:
        sample (torch.Tensor): shape (n_way, n_support+n_query, (dim))
    Returns:
        torch.Tensor: shape(2), loss, accuracy and y_hat
    """
    sample_images = sample['images'].to(device)
    n_way = sample['n_way']
    n_support = sample['n_support']
    n_query = sample['n_query']
    Classes=sample['Classes']
    Indices = sample['Index']
    x_query_Indices = Indices[:, n_support:]

    x_support = sample_images[:, :n_support]
    x_query = sample_images[:, n_support:]

    #target indices are 0 ... n_way-1
    target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
    target_inds = Variable(target_inds, requires_grad=False)
    target_inds = target_inds.to(device)

    #encode images of the support and the query set
    x = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])], 0)

    z = self.encoder.forward(x)
    z_dim = z.size(-1) #usually 64
    # Extract support and query samples
    z_samples = z[:n_way * n_support].view(n_way, n_support, z_dim)
    z_query = z[n_way * n_support:]

    # Ensure gradients are tracked
    z_samples.requires_grad_(True)
    z_query.requires_grad_(True)

    #----------------------------Making of prototypes---------------------------
    z_proto = z_samples.mean(dim=1)
    #-----------------calculationg distance from each prototypes----------------
    dists = euclidean_dist(z_query,z_proto)
    #--------------compute probabilities,loss and accuracy----------------------
    log_p_y = F.log_softmax(-dists, dim=1).view(n_way, n_query, -1)
    loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_y.max(2)
    acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()
    #-------------------------------------per class accurcay-------------
    # Compute per-class accuracy
    per_class_acc = torch.eq(y_hat, target_inds.squeeze()).float().mean(dim=1)  # Shape: (n_way,)
    #-----------------------------For softmax probabilities data--------------------------------
    softmax_probs = F.softmax(-dists, dim=1).view(n_way, n_query, -1)
    softmax_probs_flatten=torch.flatten(softmax_probs, start_dim=0, end_dim=1)
    #---------------------------Class based confidence-----------------------------------
    softmax_probs_true = softmax_probs.gather(2, target_inds)
    softmax_probs_true_squeezed = softmax_probs.gather(2, target_inds).squeeze().view(-1)

    #----------------------------misclassified points indices-----------------------------------

    target_inds_squeezed = target_inds.squeeze(-1)
    mismatches = target_inds_squeezed != y_hat
    mismatches=mismatches.cpu().data.numpy()
    Miss_selected_indices = [row[mismatches_row] for row, mismatches_row in zip(x_query_Indices, mismatches)]\
    #---------------------------------------------------------------------------------------------------------
    return loss_val, {
        'loss': loss_val.item(),
        'acc': acc_val.item(),
        'y_hat': y_hat,
        'Classes':Classes,
        'per_class_acc':per_class_acc,
        'x_query_Indices':x_query_Indices,
        'softmax_probs_true_squeezed':softmax_probs_true_squeezed,
        'target_inds':target_inds,
        'Ind':Miss_selected_indices,
        }

import numpy as np
import pandas as pd
from torch import optim
from tqdm.notebook import tnrange
from collections import Counter
def euclidean_dist(x, y):
  """
  Computes euclidean distance btw x and y
  Args:
      x (torch.Tensor): shape (n, d). n usually n_way*n_query
      y (torch.Tensor): shape (m, d). m usually n_way
  Returns:
      torch.Tensor: shape(n, m). For each query, the distances to each centroid
  """
  n = x.size(0)
  m = y.size(0)
  d = x.size(1)
  assert d == y.size(1)

  x = x.unsqueeze(1).expand(n, m, d)
  y = y.unsqueeze(0).expand(n, m, d)

  return torch.pow(x - y, 2).sum(2)
def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size,CSR,ISR,Alpha,Tau):
    """
    Trains the protonet
    Args:
        model
        optimizer
        train_x (np.array): images of training set
        train_y(np.array): labels of training set
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
        max_epoch (int): max epochs to train on
        epoch_size (int): episodes per epoch
    """
    # Divide the learning rate by 2 at each epoch, as suggested in the paper
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
    epoch = 0  # Epochs done so far
    stop = False  # Status to know when to stop

    class_set_frequency = {}  # To count how many times each class set occurs
    Tr_Samples_dict = {}  # To store sample indexes by class for each class set
    Tr_miss_dict = {}  # To store sample indexes by class for each class set

    #------------declare-------------------
    Tr_Classes=np.unique(train_y)
    accuracy_dict = {class_name: [] for class_name in Tr_Classes}
    #------------------------------------------------------------------

    # Lists to store loss and accuracy for plotting
    epoch_losses = []
    epoch_accuracies = []

    # ✅ Initialize top_vulnerable_classes for the first epoch
    top_vulnerable_classes = list(np.random.choice(np.unique(train_y), n_way, replace=False))
    while epoch < max_epoch and not stop:
        running_loss = 0.0
        running_acc = 0.0

        #---------Initialize Instance potential matrix at each epoch-------------------------------
        IP_df = Make_Instance_Potential_Matrix(train_x, train_y)
        #--------------------initialize ACS dictionary for each epoch-------------------------------
        accuracy_dict = {class_name: [] for class_name in Tr_Classes}
        # ---------------------- (ACS)Sampling strategy for each episode -------------------------
        ours_episode_count = int(epoch_size * CSR)
        sampling_strategy = [True] * ours_episode_count + [False] * (epoch_size - ours_episode_count)
        random.shuffle(sampling_strategy)
        #----------------------sampling for each episode(AIS)-----------------------------------------------
        ours_episode_count = int(epoch_size * ISR)
        sampling_strategy1 = [True] * ours_episode_count + [False] * (epoch_size - ours_episode_count)
        random.shuffle(sampling_strategy1)

        for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):

            #----------------------------ACS----------------------------------------
            # Sampling class set K
            if sampling_strategy[episode] or episode == 0:
                K = np.random.choice(np.unique(train_y), n_way, replace=False)
            else:
                K = top_vulnerable_classes
            #-----------------------------------------------------------------------
            #---------------------------AIS------------------------------------------
            if sampling_strategy1[episode] or episode == 0:
                # Use normal sampling
                sample = extract_sample_normal(n_way, n_support, n_query, train_x, train_y,K)
            else:
                # Use adaptive sampling
                sample = extract_sample_AIS(n_way, n_support, n_query, train_x, train_y,K,IP_df)
            #-----------------for visualization----------------------------------
            Tr_Classes = sample['Classes']
            Tr_Samples = sample['Index']
            Tr_Classes_tuple = tuple(sorted(Tr_Classes))
            #------------------------------------------------------------------------
            # Update class set frequency count
            if Tr_Classes_tuple in class_set_frequency:
                class_set_frequency[Tr_Classes_tuple] += 1
            else:
                class_set_frequency[Tr_Classes_tuple] = 1
                # Initialize a dictionary for each class in the class tuple
                Tr_Samples_dict[Tr_Classes_tuple] = {class_index: [] for class_index in Tr_Classes}
                Tr_miss_dict[Tr_Classes_tuple] = {class_index: [] for class_index in Tr_Classes}

            # Store sample indexes under the corresponding class and count their frequency
            for class_idx, sample_idx in zip(Tr_Classes, Tr_Samples):
                # Convert sample index to tuple (if it is a numpy array) to make it hashable
                if isinstance(sample_idx, np.ndarray):
                    sample_idx = tuple(sample_idx)
                # Add sample index to the corresponding class in the dictionary
                Tr_Samples_dict[Tr_Classes_tuple][class_idx].append(sample_idx)
            #-----------------------------------------------------------------------------
            optimizer.zero_grad()
            loss, output = model.set_forward_loss(sample)
            Tr_Miss = output['Ind']
            for class_idx, miss_idx in zip(Tr_Classes, Tr_Miss):
              if isinstance(miss_idx, np.ndarray):
                    miss_idx = tuple(miss_idx)
              Tr_miss_dict[Tr_Classes_tuple][class_idx].append(miss_idx)
            #-----------------------------------------------------------------------
            running_loss += output['loss']
            running_acc += output['acc']
            loss.backward()
            optimizer.step()
            #------------------------Dictionary of classes---------------------------------
            #----------------------------------ACS Sampling--------------------------------------
            Out_Classes=output['Classes']
            Out_per_class_acc=output['per_class_acc']
            for class_name,class_acc in zip(Out_Classes,Out_per_class_acc):
              accuracy_dict[class_name].append(class_acc)

            #-------------------------------------------------------------------------------
            #-------------------misclassified points-------------------------------------
            IP_df = calculate_mismatch_probabilities(output['softmax_probs_true_squeezed'], output['y_hat'], output['target_inds'], n_way, n_query, IP_df, output['x_query_Indices'],Alpha, Tau,output['Classes'])
        #-------------------------Calculate mean and vulnarabilities-----------------------------

        # Convert tensor values to floats
        class_probabilities_float = {cls: [p.item() for p in probs] for cls, probs in accuracy_dict.items()}
        # Compute mean probability (Γ̄(C_j)) for each class across epochs
        mean_confidences = {cls: np.mean(probs) for cls, probs in class_probabilities_float.items() if len(probs) > 0}
        # Print results
        for cls, probs in class_probabilities_float.items():
          if len(probs) > 0:  # Ensure there are valid values
            mean_confidences[cls] = np.mean(probs)
        # Sort by increasing confidence (most vulnerable = lowest confidence)
        sorted_classes = sorted(mean_confidences.items(), key=lambda x: x[1])
        top_vulnerable_classes = [cls for cls, _ in sorted_classes[:n_way]]
        #--------------------------------------------------------------------------------------
        epoch_loss = running_loss / epoch_size
        epoch_acc = running_acc / epoch_size
        print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch + 1, epoch_loss, epoch_acc))

        # Store the values for plotting
        epoch_losses.append(epoch_loss)
        epoch_accuracies.append(epoch_acc)

        epoch += 1
        scheduler.step()
    #---------------------------------------------------------------------------------------
    # After training, process and display class set frequencies
    sorted_class_sets = sorted(class_set_frequency.items(), key=lambda x: x[1], reverse=True)

    print("\nAll Class Set Frequencies:")
    for class_set, frequency in sorted_class_sets:
        print(f"Class Set: {class_set}, Frequency: {frequency}")

    print("\nTop 5 Class Sets by Frequency:")
    for class_set, frequency in sorted_class_sets[:5]:
        print(f"Class Set: {class_set}, Frequency: {frequency}")

    top_5_indexes_per_class = {}

    for class_tuple, class_samples in Tr_Samples_dict.items():
        top_5_indexes_per_class[class_tuple] = {}
        Sam=[]
        # For each class in the class tuple, count the frequency of sample indexes
        for class_idx, sample_indexes in class_samples.items():

            Sam.append([item for sublist in sample_indexes for item in sublist])

            #--------------------top 5 samples--------------------------------
            most_common_indices = [idx for idx, _ in Counter([item for sublist in sample_indexes for item in sublist]).most_common(5)]

            #-----------------------------------------------------------------

            # Store only the sample indexes, not the frequencies
            top_5_indexes_per_class[class_tuple][class_idx] = most_common_indices


    # Display the top 5 frequent sample indexes per class for each class set
    print("Top 10 frequent indexes per class:")
    print(top_5_indexes_per_class)

    top_5_miss_per_class = {}

    for class_tuple, miss_samples in Tr_miss_dict.items():
        top_5_miss_per_class[class_tuple] = {}
        Sam=[]
        # For each class in the class tuple, count the frequency of sample indexes
        for class_idx, miss_indexes in miss_samples.items():

            Sam.append([item for sublist in miss_indexes for item in sublist])

            #--------------------top 5 samples--------------------------------
            most_common_indices = [idx for idx, _ in Counter([item for sublist in miss_indexes for item in sublist]).most_common(10)]

            #-----------------------------------------------------------------

            # Store only the sample indexes, not the frequencies
            top_5_miss_per_class[class_tuple][class_idx] = most_common_indices


    # Display the top 5 frequent sample indexes per class for each class set
    print("Top 10 frequent indexes per class in missclassified:")
    print(top_5_miss_per_class)

    for class_pair, classes in top_5_indexes_per_class.items():
      print(f"\nClass Pair: {class_pair}")
      for cls, elements in classes.items():
        print("cls",cls)
        print("elements",elements)
        if is_non_empty(elements) == True:
          datax_cls = train_x[train_y == cls]
          sample_cls = datax_cls[elements]
          print("************For class:**************",cls)


          heights = [img.shape[0] for img in sample_cls]
          if len(set(heights)) > 1:
              raise ValueError("All images must have the same height for a horizontal grid.")

          # Concatenate images horizontally
          sample_cls_rgb = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in sample_cls]
          grid_image = np.concatenate(sample_cls_rgb, axis=1)  # Concatenate along width

          plt.figure(figsize=(15, 5))
          # Display the resulting grid
          plt.imshow(grid_image)
          plt.axis('off')
          #plt.title("*********Images for Class : {}**********".format(cls), fontsize=16)  # Add title
          plt.show()

    #---------------------------------------------------------------------------
def find_common_elements(dict1, dict2):
    common_elements = {}

    for key in dict1:
        if key in dict2:
            class1, class2 = key
            common_elements[key] = {}
            for subkey in dict1[key]:
                # Get the lists from both dictionaries
                list1 = dict1[key][subkey]
                list2 = dict2[key][subkey]

                # Find elements that are the same in both lists
                common_elements[key][subkey] = [list1[i] for i in range(len(list1)) if list1[i] == list2[i]]

    return common_elements
def is_non_empty(lst):
    return bool(lst)
def test(model, test_x, test_y, n_way, n_support, n_query, test_episode):
    """
    Tests the protonet
    Args:
        model: trained model
        test_x (np.array): images of testing set
        test_y (np.array): labels of testing set
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
        test_episode (int): number of episodes to test on
    """
    model.eval()

    loss_list = []
    acc_list = []

    running_loss = 0.0
    running_acc = 0.0
    for episode in range(test_episode):
        K = np.random.choice(np.unique(test_y), n_way, replace=False)
        sample = extract_sample_normal(n_way, n_support, n_query, test_x, test_y,K)
        loss, output = model.set_forward_loss(sample)

        running_loss += output['loss']
        running_acc += output['acc']

        # Append the current loss and accuracy for later plotting
        loss_list.append(output['loss'])
        acc_list.append(output['acc'])

    avg_loss = running_loss / test_episode
    avg_acc = running_acc / test_episode
    print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))
    mean_accuracy,margin_of_error=Calc_Conf_Acc(acc_list)
    return mean_accuracy,margin_of_error
def Calc_Conf_Acc(Calc_Test_Conf):
  flattened_Calc_Test_Conf = Calc_Test_Conf

  mean_accuracy = np.mean(flattened_Calc_Test_Conf)

  std_dev = np.std(flattened_Calc_Test_Conf)

  # Calculate standard error of the mean
  std_error = stats.sem(flattened_Calc_Test_Conf)

  # Calculate margin of error for 95% confidence interval
  margin_of_error = stats.t.ppf(0.975, len(flattened_Calc_Test_Conf)-1) * std_error

  # Calculate lower and upper bounds of the confidence interval
  lower_bound = mean_accuracy - margin_of_error
  upper_bound = mean_accuracy + margin_of_error
  return mean_accuracy,margin_of_error



#-----------Main function-----------------------------------------------------

n_way = 2
n_support = 5
n_query = 5
epochs_to_run = [2]
epoch_size = 2
test_episode = 6
Size=224
#-------------------------set seeeds-----------------------------
np_seed = 0 # Set the seed for reproducibility
torch_seed = 0
CSR=0.2
ISR=0.5

# Fix all seeds
np.random.seed(np_seed)
torch.manual_seed(torch_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(torch_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
Alpha=0.5
Tau=0.3
# Load pickle files
PATH = '/content/gdrive/MyDrive/Refined-Pickle-Files/Refined-Pickle-Files/Pickle-'+str(Size)+'_'+str(Size)+'/'
Data='Derm7pt'

with open(PATH + Data + "_trainx.pkl", "rb") as f:
    trainx = pickle.load(f)
with open(PATH + Data + "_trainy.pkl", "rb") as f:
    trainy = pickle.load(f)
with open(PATH + Data + "_testx.pkl", "rb") as f:
    testx = pickle.load(f)
with open(PATH + Data + "_testy.pkl", "rb") as f:
    testy = pickle.load(f)

# Function to track the highest accuracy
def get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode):
    # Assuming this function is available in your framework
    return test(model, testx, testy, n_way, n_support, n_query, test_episode)

best_accuracy = 0
best_epoch = 0

# Training and testing for specified epochs
for epoch in epochs_to_run:
    # Initialize model and optimizer
    model = load_protonet_conv(x_dim=(3, Size, Size), hid_dim=64, z_dim=64)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"-----------------------Training for {epoch} epochs-----------------------")
    train(model, optimizer, trainx, trainy, n_way, n_support, n_query, epoch, epoch_size,CSR,ISR,Alpha, Tau)

    print(f"-----------------------Testing after {epoch} epochs-----------------------")
    mean_accuracy,margin_of_error = get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode)

    print(f"Accuracy after {epoch} epochs: {mean_accuracy}")

    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_epoch = epoch
        best_mar=margin_of_error

print(f"\nHighest accuracy: {best_accuracy} and error margin: {best_mar} achieved at epoch: {best_epoch}")


-----------------------Training for 2 epochs-----------------------


Epoch 1 train:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1 -- Loss: 36.4220 Acc: 0.5500


Epoch 2 train:   0%|          | 0/2 [00:00<?, ?it/s]

KeyError: np.str_('vascular lesion')